In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import urllib.parse
import seaborn as sns
from tqdm import tqdm
import src.data.data_loader as data_loader

pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [ ]:
all_paths, paths_finished, paths_unfinished = data_loader.load_paths()
articles = data_loader.load_articles()
categories = data_loader.load_categories()
links = data_loader.load_links()
distance_matrix = data_loader.load_distance_matrix()


# Frustration

Our initial approach to measuring player frustration focuses on analyzing instances of backtracking. When players need to retrace their steps within a game, it often signals an increase in difficulty, which can be directly linked to frustration. Let's dive into it :

In [ ]:
print(f"Across the study, we have {len(paths_finished)} finished paths, while {len(paths_unfinished)} remain unfinished, for a total of {len(all_paths)} paths.")


In [ ]:
# Calculate the average number of backtracks for finished vs unfinished paths
avg_backtracks_finished = all_paths[all_paths['status'] == 'finished']['backtracks'].apply(len).mean()
avg_backtracks_unfinished = all_paths[all_paths['status'] != 'finished']['backtracks'].apply(len).mean()

# Output the results
print("\nAverage number of backtracks per game:")
print(f"Finished paths: {avg_backtracks_finished:.2f}")
print(f"Unfinished paths: {avg_backtracks_unfinished:.2f}")


In [ ]:
# Get finished games with and without backtracks

finished_with_bt = paths_finished[paths_finished['backtracks'].apply(len) > 0]
finished_no_bt = paths_finished[paths_finished['backtracks'].apply(len) == 0]

# Calculate average duration for each group
avg_duration_with_bt = finished_with_bt['durationInSec'].median()
avg_duration_no_bt = finished_no_bt['durationInSec'].median()

# Group finished games by number of backtracks and calculate median duration
backtrack_counts = paths_finished.groupby(paths_finished['backtracks'].apply(len)).size()
backtrack_durations = paths_finished.groupby(paths_finished['backtracks'].apply(len))['durationInSec'].median()

# Filter to keep only counts with at least 10 samples
filtered_counts = backtrack_counts[backtrack_counts >= 10]
filtered_durations = backtrack_durations[filtered_counts.index]

# Create the plot
plt.figure(figsize=(10,6))
plt.plot(filtered_durations.index, filtered_durations.values, marker='o')
plt.axvline(x=avg_backtracks_finished, color='r', linestyle='--', label='Average backtracks')
plt.axhline(y=avg_duration_with_bt, color='g', linestyle='--', label='Average duration')
plt.xlabel('Number of Backtracks')
plt.ylabel('Median Duration (seconds)')
plt.title('Median Game Duration vs Number of Backtracks (10+ samples)')
plt.grid(True)

# Add a trend line
z = np.polyfit(filtered_durations.index, filtered_durations.values, 1)
p = np.poly1d(z)
plt.plot(filtered_durations.index, p(filtered_durations.index), "r--", alpha=0.8, 
         label=f'Trend line (slope: {z[0]:.1f})')

# Calculate R-squared
r_squared = r2_score(filtered_durations.values, p(filtered_durations.index))

plt.legend()
plt.text(0.05, 0.95, f'R-squared: {r_squared:.4f}', transform=plt.gca().transAxes, 
         verticalalignment='top')
plt.show()

print("Counts of games with each number of backtracks:")
print(filtered_counts)
print(f"\nR-squared value: {r_squared:.4f}")



In today’s fast-paced world, where content is rapidly consumed, players’ attention spans are increasingly shorter. If a game drags on too long, frustration can build as players wait longer for the rewarding sense of completion. As illustrated in the graph above, there is a noticeable correlation between the frequency of backtracking and overall game duration: each backtrack adds an average of 43 seconds to the game time.

In [ ]:
# Group finished games by number of backtracks and calculate median duration
backtrack_counts = paths_finished.groupby(paths_finished['backtracks'].apply(len)).size()
backtrack_ratings = paths_finished.groupby(paths_finished['backtracks'].apply(len))['rating'].mean()

# Filter to keep only counts with at least 10 samples
filtered_counts = backtrack_counts[backtrack_counts >= 10]
filtered_ratings = backtrack_ratings[filtered_counts.index]

# Create the plot
plt.figure(figsize=(10,6))
plt.plot(filtered_ratings.index, filtered_ratings.values, marker='o')
plt.xlabel('Number of Backtracks')
plt.ylabel('Average Rating')
plt.axvline(x=avg_backtracks_finished, color='r', linestyle='--', label='Average backtracks')
plt.axhline(y=paths_finished['rating'].mean(), color='g', linestyle='--', label='Average rating')
plt.title('Average Rating vs Number of Backtracks (10+ samples)')
plt.grid(True)

# Add a trend line
z = np.polyfit(filtered_ratings.index, filtered_ratings.values, 1)
p = np.poly1d(z)

plt.plot(filtered_ratings.index, p(filtered_ratings.index), "r--", alpha=0.8, 
         label=f'Trend line (slope: {z[0]:.1f})')

# Calculate R-squared
r_squared = r2_score(filtered_ratings.values, p(filtered_ratings.index))

plt.legend()
plt.text(0.05, 0.95, f'R-squared: {r_squared:.4f}', transform=plt.gca().transAxes, 
         verticalalignment='top')
plt.show()

print("Counts of games with each number of backtracks:")
print(filtered_counts)
print(f"\nR-squared value: {r_squared:.4f}")

In [ ]:
# Create mask for paths with backtracking and valid ratings
has_backtrack_mask = paths_finished['path'].str.contains('<')
has_rating_mask = paths_finished['rating'].notna()

# Filter ratings into two groups
ratings_with_backtrack = paths_finished.loc[has_backtrack_mask & has_rating_mask, 'rating']
ratings_without_backtrack = paths_finished.loc[~has_backtrack_mask & has_rating_mask, 'rating']

# Calculate averages using pandas methods
average_back = ratings_with_backtrack.mean()
average_no_back = ratings_without_backtrack.mean()


# Plot the histograms
plt.figure(figsize=(10, 6))

# Plot histograms
plt.hist(ratings_with_backtrack, bins=5, color='orange', edgecolor='black', alpha=0.7, label='With Backtracking')
plt.hist(ratings_without_backtrack, bins=5, color='skyblue', edgecolor='black', alpha=0.7, label='Without Backtracking')

# Add vertical lines for the averages
plt.axvline(x=average_back, color='red', linestyle='-', linewidth=2, label=f'Average (with backtracking) = {average_back:.2f}')
plt.axvline(x=average_no_back, color='blue', linestyle='-', linewidth=2, label=f'Average (without backtracking) = {average_no_back:.2f}')

# Add labels and title
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.xticks(range(1,6))
plt.title('Histogram of Ratings for Paths With and Without Backtracking')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()


One other indicator of player frustration we consider is the game’s "difficulty rating" assigned by the player upon completion. Generally, as the difficulty of a game increases, so does the rating, offering a potential correlation with the level of frustration experienced. If the player needs to backtrack, the perceived difficulty of the game seems higher.

In [ ]:

# Create a DataFrame with counts of backtracks and traversed
backtracks = pd.Series(all_paths['backtracks'].explode().dropna().tolist())
traversed = pd.Series(all_paths['traversed'].explode().dropna().tolist())

word_counts = pd.DataFrame({
    'backtracks': backtracks.value_counts(),
    'traversed': traversed.value_counts()
})

# Calculate the ratio of backtracks to traversed
word_counts['backtrack_ratio'] = word_counts['backtracks'] / word_counts['traversed']

# Fill NaN values with 0 (for words that were traversed but never backtracked)
word_counts['backtrack_ratio'] = word_counts['backtrack_ratio'].fillna(0)

# Sort by the ratio in descending order
word_counts_sorted = word_counts.sort_values('backtrack_ratio', ascending=False)

# Display the top 20 words with highest backtrack ratios for words with at least 100 occurrences
print(word_counts_sorted[word_counts_sorted['traversed'] >= 0].head(20))

# Optional: Plot the distribution of backtrack ratios
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.hist(word_counts['backtrack_ratio'], bins=50, edgecolor='black')
plt.title('Distribution of Backtrack Ratios')
plt.xlabel('Backtrack Ratio')
plt.ylabel('Frequency')
plt.show()



In [ ]:
paths_unfinished.groupby('type').count()

In [ ]:
# Count the number of games in each 'type' group
game_counts_by_type = paths_unfinished.groupby('type').size()

# Display the result
print(game_counts_by_type)


In conclusion, it’s evident that the game induces a certain level of frustration for players, with our backtracking analysis supporting this observation. The next step is to delve into understanding the causes of this frustration, particularly identifying what aspects of the game design lead to the need for backtracking.

---

# Explanation

In [ ]:
# Create DataFrame of backtrack-target pairs
backtrack_df = pd.DataFrame([
    {'backtrack': backtrack, 'target': row['target']}
    for _, row in all_paths[all_paths['backtracks'].apply(len) > 0][['backtracks','target']].iterrows()
    for backtrack in row['backtracks']
])

print("\nArticle of backtrack vs target")
print(backtrack_df.value_counts().head(20))


We’ve observed that numerous backtracks stem from confusion over closely related words or concepts. Could this reveal gaps or missing connections within the game’s design that prevent players from progressing/finishing?

In [ ]:
# Find missing targets 
articles_set = set(articles['article'])
missing_targets = set([target for target in paths_unfinished['target'] if target not in articles_set])

if missing_targets:
    print("Missing targets:")
    for target in missing_targets:
        print(target)
else:
    print("No missing targets found.")


Indeed, there are missing connections—and, even more frustratingly, some destinations are entirely unreachable. Imagine the player’s frustration in searching for “Christmas,” only to complete the game on “Santa Claus” and realize there’s no link back to “Christmas.” This lack of connection not only disrupts gameplay but can also leave players feeling stuck and unsatisfied, amplifying their frustration.

---

# Solution

To assess if a link is needed between two articles and measure their "distance," we can use:

- **Shortest Path**: Count the hops between articles in the dataset. A high hop count suggests a possible need for a direct link.
- **Cosine Similarity**: Calculate similarity between article embeddings. High similarity without a link may indicate a need for connection.
The heatmap below shows the differences between these two methods : 

In [ ]:
# Step 1: Replace -1 values with np.nan to ignore them during normalization
distance_matrix_normalized = np.where(distance_matrix == -1, np.nan, distance_matrix)

# Step 2: Perform normalization on the matrix, excluding NaN values
min_value = np.nanmin(distance_matrix_normalized)
max_value = np.nanmax(distance_matrix_normalized)
distance_matrix_normalized = (distance_matrix_normalized - min_value) / (max_value - min_value)

# Step 3: Replace NaNs (original -1 values) with 1
distance_matrix_normalized = np.where(np.isnan(distance_matrix_normalized), 1, distance_matrix_normalized)


min_value = np.nanmin(distance_matrix_normalized)
max_value = np.nanmax(distance_matrix_normalized)
distance_matrix_normalized = (distance_matrix_normalized - min_value) / (max_value - min_value)

# Step 3: Replace NaNs (original -1 values) with 1
distance_matrix_normalized = np.where(np.isnan(distance_matrix_normalized), 1, distance_matrix_normalized)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(distance_matrix_normalized, cmap="YlGnBu", annot=False, cbar=True)
plt.title('Heatmap of Normalized Distance Matrix')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()


 ### Choice of similarity model

Let's look at that game, it will also enable us to select which model we want to use for the rest of the analysis to compute the similarity between 2 articles :

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from src.utils.similarity import get_cosine_similarity_for_path_word_by_word, get_cosine_similarity_for_path_word_by_word_openai
from src.utils.utils import path_to_list

sb_model = SentenceTransformer("all-MiniLM-L6-v2", device="mps" if torch.backends.mps.is_available() else "cpu")
clip_model = SentenceTransformer("clip-ViT-B-32", device="mps" if torch.backends.mps.is_available() else "cpu")

christmas_path="Bob_Dylan;England;Church_of_England;Christianity;Jesus;Trinity;Christianity;Bible;God;United_States;Turkey_%28bird%29;Domesticated_turkey;A_Christmas_Carol;Bob_Marley;Ethiopia;Meat;Chicken;Poultry;Food;Season;Winter;Snow;Bonobo;Human;Culture;Ice_age;Earth;Middle_Ages;Holy_Roman_Empire;Switzerland;World_War_I;Finland;Arctic_Circle;Greenland;Iceland;Denmark;German_language;Hebrew_language;Rome;Roman_Catholic_Church;Anglicanism;Roman_Britain;Gold;Olympic_Games;1896_Summer_Olympics;Austria;World_War_II;Jew;Religion;Sun;Sunlight;<;Solar_eclipse;Planet;Saturn;Zeus;Greek_mythology;Agriculture;<;Apollo;Roman_mythology;Ancient_Rome;Christianity;Jesus;Last_Supper;Wine;Alcohol;<;Nutmeg;Tree;Chestnut;Bread;Europe;Russia;Lithuania;Poland;Potato;Dill;Soup;Egg_%28food%29;Food;Advertising;<;Pottery;Coal;Santa_Claus"
christmas_path = path_to_list(christmas_path)
bert_similarities = get_cosine_similarity_for_path_word_by_word(christmas_path, "Christmas", sb_model)
clip_similarities = get_cosine_similarity_for_path_word_by_word(christmas_path, "Christmas", clip_model)
openai_similarities = get_cosine_similarity_for_path_word_by_word_openai(christmas_path, "Christmas")


plt.figure(figsize=(18, 8))
plt.plot(range(len(christmas_path)), bert_similarities, marker='o', label='SentenceBERT')
plt.plot(range(len(christmas_path)), clip_similarities, marker='o', label='CLIP')
plt.plot(range(len(christmas_path)), openai_similarities, marker='o', label='OpenAI')
plt.title('Path to Christmas')
plt.xlabel('Step in Path') 
plt.ylabel('Cosine Similarity')
plt.grid(True)
plt.legend()

# Add path words as x-tick labels
plt.xticks(range(len(christmas_path)), christmas_path, rotation=45, ha='right')
plt.tight_layout()
plt.show()


After a thorough and meticulous analysis of various models—including SentenceBERT, CLIP, and OpenAI— We selected the OpenAI model for its superior accuracy and consistency in delivering precise results through cosine similarity.

### First test of threshold to find missing links : Fixed threshold on the similarity

In [ ]:
from src.utils.similarity import should_have_link

#using arbitrary threshold
print(should_have_link("Santa Claus","Christmas"))
print(should_have_link("Santa Claus","Summer"))
print(should_have_link("Soybean","Bean"))


In [ ]:
# Find potential missing links between articles based on semantic similarity
should_link_dict = {}
missing_link_pairs = []

# Iterate through all articles
for source_article in tqdm(articles['article'][:100]):#only 100 articles to limit runtime
    # Get existing links from current article
    existing_links = links[links['linkSource'] == source_article]
    full_source_article = open(f"data/plaintext_articles/{urllib.parse.quote(source_article.replace(' ', '_'))}.txt","r").read()
    source_article_lowered = full_source_article.lower()

    # Find articles that aren't currently linked from source article
    potential_targets = articles[~articles['article'].isin(existing_links['linkTarget'])]
    potential_targets = potential_targets[potential_targets['article'].str.lower().apply(lambda x: x in source_article_lowered)]
    potential_targets = potential_targets[potential_targets['article'].apply(lambda x: should_have_link(source_article,x)[0])]
    potential_targets = potential_targets[potential_targets['article'] != source_article]
    
    
    if len(potential_targets) > 0:
        should_link_dict[source_article]= potential_targets['article'].tolist()
        for target in potential_targets['article']:
            missing_link_pairs.append({
                'source': source_article,
                'target': target
            })


for article,targets in should_link_dict.items():
    print(f"{article}: {targets}")

pd.DataFrame(missing_link_pairs).to_csv('data/missing_links.csv', index=False)

In this initial iteration, the model appears to perform well, allowing us to easily add additional links. It identifies article titles mentioned within the text that are not yet linked, even when they are semantically close. 

---

# Is there a better way to do it ?

Let’s analyze the similarities across the entire game by calculating the similarity between each page and the target article.

In [ ]:
all_path_similarities_unfinished, all_path_similarities_finished = data_loader.load_all_path_similarities()

In [ ]:
# Take only the last 10 elements of each path
from src.utils.utils import shorten_list_finished, shorten_list_unfinished

all_path_similarities_unfinished_copy = all_path_similarities_unfinished.copy()
all_path_similarities_unfinished_copy['similarities'],all_path_similarities_unfinished_copy['path'] = zip(*all_path_similarities_unfinished_copy.apply(shorten_list_unfinished_, axis=1))

similarities_unfinished = all_path_similarities_unfinished_copy[all_path_similarities_unfinished_copy['similarities'].apply(lambda x: len(x) > 0)][['similarities','path','target']]


all_path_similarities_finished_copy = all_path_similarities_finished.copy()
all_path_similarities_finished_copy['similarities'],all_path_similarities_finished_copy['path'] = zip(*all_path_similarities_finished_copy.apply(shorten_list_finished, axis=1))
similarities_finished = all_path_similarities_finished_copy['similarities']

In [ ]:

# Create a figure with subplots for the first 10 paths
fig, axes = plt.subplots(5, 2, figsize=(15, 25))
axes = axes.flatten()

# Colors for each model
colors = {
    'similarities': 'green'
}

selected_finished = all_path_similarities_finished_copy.sample(10)
# Plot first 10 paths
for i in range(10):
    path = selected_finished.apply(lambda x: x['path'],axis=1).iloc[i]
    target = selected_finished['target'].iloc[i]
    
    # Plot each model's similarities
    for model in ['similarities']:
        similarities = selected_finished.apply(lambda x: x[model],axis=1).iloc[i]
        axes[i].plot(range(len(similarities)), similarities, 
                    marker='o', label=model, color=colors[model])
    
    # Customize the plot
    axes[i].set_title(f'Path {i+1}\nTarget: {target}')
    axes[i].set_xlabel('Step in Path')
    axes[i].set_ylabel('Cosine Similarity')
    axes[i].set_ylim(0, 1)
    axes[i].grid(True)
    axes[i].legend()
    
    # Add path words as x-tick labels
    axes[i].set_xticks(range(len(path)))
    axes[i].set_xticklabels(path, rotation=45, ha='right')

plt.tight_layout()
plt.show()


Mhhhhh, it seems like there is a trend over there !!

# Plot similarity evolution for 100 paths together

In [ ]:
# Create figure and axis
plt.figure(figsize=(15, 8))

# Plot first 100 similarity evolutions
random_finished = all_path_similarities_finished_copy.sample(100)
for index, path in random_finished.iterrows():
    similarities = path['similarities']
    reversed_similarities = similarities[::-1]
    plt.plot(reversed_similarities)


plt.xlabel('distance to end of game')
plt.ylabel('Similarity Score') 
plt.xlim(9,0)
plt.title('Evolution of Similarities for First 100 Paths')

# Add grid for better readability
plt.grid(True)

plt.show()


Across all completed games, we observe a clear trend in the endgame stage. Let’s examine whether this trend extends to unfinished games as well (excluding the final step, of course).

In [ ]:
from src.utils.similarity import process_similarities

similarities_by_distance_finished = process_similarities(similarities_finished)
similarities_by_distance_unfinished = process_similarities(similarities_unfinished['similarities'])

In [ ]:
# Calculate means, medians, and standard deviations
stats = {
    'Finished': [similarities_by_distance_finished, range(10)],
    'Unfinished': [similarities_by_distance_unfinished, range(1,10)]
}

plt.figure(figsize=(15, 8))
colors = ['#1f77b4', '#2ca02c']

for i, (path_type, (similarities, x)) in enumerate(stats.items()):
    means = [np.mean(sublist) for sublist in similarities]
    stds = [np.std(sublist) for sublist in similarities]
    
    plt.plot(x, means, color=colors[i], label=f'Mean ({path_type})', linewidth=2)
    plt.fill_between(x, [m - s for m, s in zip(means, stds)],
                     [m + s for m, s in zip(means, stds)],
                     color=colors[i], alpha=0.2)

plt.xlabel('Distance from End', fontsize=12)
plt.ylabel('Average Similarity Score', fontsize=12)
plt.ylim(0,1)
plt.xlim(9,0)
plt.title('Mean Similarity Score vs Distance from End', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()


Our analysis confirms a trend in the endgame stage, even within unfinished paths. To investigate this, we focused on the highest similarity point within each game and the last eight moves leading up to it. This trend reveals several possible scenarios:

- **Player Confusion**: Some players appear to struggle, failing to reach a similarity threshold of 0.3. In such cases, the game might be disregarded by the search algorithm, as the low similarity indicates a lack of meaningful progression.

- **Missed Links**: Players may overlook critical links if they appear later in the article rather than at the beginning. This suggests a potential improvement area in link placement to aid player navigation. (Refer to the accompanying graph for a visual representation.)

- **Missing Final Link**: In some cases, there may be a missing link between the last article the player reached and the target, creating a gap in progression and heightening player frustration.

In [ ]:
chosen_links_df =pd.read_csv('data/chosen_links_rank.csv')


rank_data = chosen_links_df['RankChosen'].dropna()

# Count the frequency of each rank
rank_counts = rank_data.value_counts()

plt.figure(figsize=(10, 6))
plt.bar(rank_counts.index, rank_counts.values, color='skyblue')
plt.xlabel('Position of Chosen Link in Article')
plt.xlim(1, 30)  # Limit to first 20 positions
plt.ylabel('Frequency')
plt.title('Distribution of Chosen Links Based on Position in Article')

# Set x-ticks to show every position within the limited range
plt.xticks(range(1, 31))  # Adjust as necessary

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


---

# Find missing link with our new method

In [ ]:
from src.utils.similarity import get_missing_links

unfinished_missing_links = similarities_unfinished.apply(get_missing_links,axis=1)
print("Most common pairs of articles that may be missing links:")
print(unfinished_missing_links.value_counts().head(20))


In [ ]:
# Filter pairs that appear at least 2 times
frequent_pairs = unfinished_missing_links.value_counts()[unfinished_missing_links.value_counts() >= 2]
link_missed_by_player = []
link_absent = []

for source, target in frequent_pairs.index:
    full_source_article = open(f"data/plaintext_articles/{urllib.parse.quote(source.replace(' ', '_'))}.txt","r").read()
    source_article_lowered = full_source_article.lower()
    if links[(links['linkSource'] == source) & (links['linkTarget'] == target)].shape[0] > 0:
        link_missed_by_player.append((source,target))
    elif target.lower() in source_article_lowered:
        link_absent.append((source,target))


print(f"Link missed by player: {len(link_missed_by_player)}")
print(f"Link absent: {len(link_absent)}")

# Save missing links to CSV
missing_links_df = pd.DataFrame(link_absent, columns=['source', 'target'])
missing_links_df.to_csv('data/missing_links.csv', index=False)

# Save links missed by players to CSV 
missed_by_player_df = pd.DataFrame(link_missed_by_player, columns=['source', 'target'])
missed_by_player_df.to_csv('data/links_missed_by_player.csv', index=False)





Now we can happily classify each of the unfinished game :)

We have officially saved Christmas, allowing Santa Claus to join the party ! 